In [4]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [6]:
import connectorx as cx
import os
import pandas as pd

import src.data.utils as du

# Deployed lookup testing

Using this to both refine lookup speed and boilerplate code for achieving our target tasks:

* Joining multiple tables with duplicates in the source and targets
* Joining multiple tables with duplicates in the source, but the best match in the target
* Finding duplicates in the source

It's worth noting the below is just for raw data extraction. Especially when duplicates are involved, you'd expect the below to become aggregation queries, which will slow stuff up.

## With dupes in both source and target

Joining data from three medium-sized tables.

V1 index is two multicolumn b-tree indices on `source`/`target` and `source_id`/`target_id`.

| `return_type` | lib | index on lookup | time (seconds) |
| --- | --- | --- | --- |
| None | pgAdmin | no | 53 |
| `arrow` | `connectorx` | no | 52 |
| `pandas` | `connectorx` | no | 115 |
| `pandas` | `pandas=1.3.5` | no | 72 |
| None | pgAdmin | v1 | 15 |
| `arrow` | `connectorx` | v1 | 10 |
| `pandas` | `connectorx` | v1 | 15 |
| `pandas` | `pandas=1.3.5` | v1 | 17 |

In [4]:
sql = """
    select
        ch.id,
        ch.company_name as ch_name,
        dh.name as dh_name,
        ew.company_name as ew_name
    from (
        select 
            *
        from
            _user_eaf4fd9a.lookup lookup
        where
            lookup.source = 'companieshouse_companies'
            and lookup.target in (
                'dit_data_hub__companies',
                'dit_export_wins__wins_dataset'
            )
    ) lookup
    right outer join companieshouse.companies ch on
        lookup.source_id = ch.id::text
        and lookup.source = 'companieshouse_companies'
    left join dit.data_hub__companies dh on
        lookup.target_id = dh.id::text
        and lookup.target = 'dit_data_hub__companies'
    left join dit.export_wins__wins_dataset ew on
        lookup.target_id = ew.id::text
        and lookup.target = 'dit_export_wins__wins_dataset'  
"""

In [9]:
%%time

df = cx.read_sql(
    conn = f"postgres://{os.environ['PGUSER']}:{os.environ['PGPASSWORD']}@{os.environ['PGHOST']}:{os.environ['PGPORT']}/{os.environ['PGDATABASE']}",
    query = sql,
    return_type = "arrow"
)

CPU times: user 5.07 s, sys: 375 ms, total: 5.44 s
Wall time: 10.6 s


In [6]:
%%time

df = cx.read_sql(
    conn = f"postgres://{os.environ['PGUSER']}:{os.environ['PGPASSWORD']}@{os.environ['PGHOST']}:{os.environ['PGPORT']}/{os.environ['PGDATABASE']}",
    query = sql,
    return_type = "pandas"
)

CPU times: user 5.39 s, sys: 673 ms, total: 6.07 s
Wall time: 17.3 s


In [7]:
%%time

with du.sql_engine.connect() as connection:
    df = pd.read_sql(
        sql, 
        connection
    )

CPU times: user 10.7 s, sys: 1.44 s, total: 12.2 s
Wall time: 19 s


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5379033 entries, 0 to 5379032
Data columns (total 4 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   id       object
 1   ch_name  object
 2   dh_name  object
 3   ew_name  object
dtypes: object(4)
memory usage: 164.2+ MB


## Without dupes in target, with dupes in source

Top result only from targets, ignores duplicates in source. Same data as above. No non-indexed benchmark, sorry.

When using a deduped source, this will result in unique source to top result in target.

V1 index is two multicolumn b-tree indices on `source`/`target` and `source_id`/`target_id`.

| `return_type` | lib | index on lookup | time (seconds) |
| --- | --- | --- | --- |
| None | pgAdmin | v1 | 27 |
| `arrow` | `connectorx` | v1 | 12 |
| `pandas` | `connectorx` | v1 | 22 |
| `pandas` | `pandas=1.3.5` | v1 | 21 |

In [2]:
sql = """
    select
        ch.id,
        ch.company_name as ch_name,
        dh.name as dh_name,
        ew.company_name as ew_name
    from (
        select
            companieshouse_companies,
            max(dit_data_hub__companies) as dit_data_hub__companies,
            max(dit_export_wins__wins_dataset) as dit_export_wins__wins_dataset
        from crosstab(
            'select distinct on (target, target_id)
                source_id,
                target,
                target_id
            from (
                select distinct on (source_id, target)
                    source_id,
                    target,
                    target_id,
                    match_probability
                from
                    "_user_eaf4fd9a"."lookup" lookup
                where
                    lookup.source = ''companieshouse_companies''
                    and lookup.target in (
                        ''dit_data_hub__companies'',
                        ''dit_export_wins__wins_dataset''
                    )
                order by
                    source_id,
                    target,
                    target_id,
                    match_probability desc
            ) lookup
            order by
                target, 
                target_id,
                match_probability',
            $$ values
                ('dit_data_hub__companies'::text), 
                ('dit_export_wins__wins_dataset'::text)
            $$ 
        ) as ct (
            "companieshouse_companies" text,
            "dit_data_hub__companies" text,
            "dit_export_wins__wins_dataset" text
        )
        group by
            companieshouse_companies
    ) lookup
    right outer join companieshouse.companies ch on
        lookup.companieshouse_companies = ch.id::text
    left join dit.data_hub__companies dh on
        lookup.dit_data_hub__companies = dh.id::text
    left join dit.export_wins__wins_dataset ew on
        lookup.dit_export_wins__wins_dataset = ew.id::text
"""

In [7]:
%%time

df = cx.read_sql(
    conn = f"postgres://{os.environ['PGUSER']}:{os.environ['PGPASSWORD']}@{os.environ['PGHOST']}:{os.environ['PGPORT']}/{os.environ['PGDATABASE']}",
    query = sql,
    return_type = "arrow"
)

CPU times: user 3.61 s, sys: 192 ms, total: 3.81 s
Wall time: 12.4 s


In [8]:
%%time

df = cx.read_sql(
    conn = f"postgres://{os.environ['PGUSER']}:{os.environ['PGPASSWORD']}@{os.environ['PGHOST']}:{os.environ['PGPORT']}/{os.environ['PGDATABASE']}",
    query = sql,
    return_type = "pandas"
)

CPU times: user 4.07 s, sys: 579 ms, total: 4.65 s
Wall time: 22.4 s


In [9]:
%%time

with du.sql_engine.connect() as connection:
    df = pd.read_sql(
        sql, 
        connection
    )

CPU times: user 9.6 s, sys: 1.02 s, total: 10.6 s
Wall time: 21 s


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5359637 entries, 0 to 5359636
Data columns (total 4 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   id       object
 1   ch_name  object
 2   dh_name  object
 3   ew_name  object
dtypes: object(4)
memory usage: 163.6+ MB


## Finding duplicates in source

V1 index is two multicolumn b-tree indices on `source`/`target` and `source_id`/`target_id`.

| `return_type` | lib | index on lookup | time (seconds) |
| --- | --- | --- | --- |
| None | pgAdmin | v1 | x |
| `arrow` | `connectorx` | v1 | x |
| `pandas` | `connectorx` | v1 | x |
| `pandas` | `pandas=1.3.5` | v1 | x |